# ДЗ 3

Домашнее задание после из лекции 3

Используя RandomForestClassifier и еще 2 метода МО предсказать будущий ABC класс на основе любых фичей

#### Путь к датасету:

In [26]:
FILEPATH = os.path.join("DataSet_w_NA.xlsx")

#### ABC-анализ по алгоритму, представленному в лекции:

In [43]:
import pandas as pd
import os

In [44]:
df = pd.read_excel(FILEPATH, sheet_name="Испорченные факты")

In [19]:
df.head()

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.20,1,7,3
1,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.80,1,4,8
2,2014-100328,Нет,2015-07-28 00:00:00,2015-08-03 00:00:00,Стандарт,CLNT0000342,PROD0000187,1964.0,1.0,0.2,662.85,1,4,12
3,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001095,1184.0,2.0,0.2,414.40,2,12,7
4,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001616,9504.0,3.0,0.2,3445.20,2,12,3


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10293 entries, 0 to 10292
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Номер заказа            10293 non-null  object 
 1   Возврат                 10293 non-null  object 
 2   Дата заказа             10293 non-null  object 
 3   Дата отправки           10293 non-null  object 
 4   Режим отправки          10293 non-null  object 
 5   Факты.Контрагент ID     10203 non-null  object 
 6   Факты.Товар ID          10242 non-null  object 
 7   Продажи, руб            10098 non-null  float64
 8   Продажи, шт             10272 non-null  float64
 9   Скидка                  10293 non-null  float64
 10  Маржинальная прибыль    10275 non-null  float64
 11  Повторение заказа       10293 non-null  int64  
 12  Повторение контрагента  10293 non-null  int64  
 13  Повторение товара       10293 non-null  int64  
dtypes: float64(4), int64(3), object(7)
mem

In [21]:
df.isna().sum().sum()

375

In [45]:
df = df.dropna()

In [46]:
df_sales =pd.pivot_table(df, values=['Продажи, руб', 'Продажи, шт','Повторение заказа','Маржинальная прибыль','Повторение товара'], index=["Факты.Товар ID"],
                     aggfunc={'Продажи, шт': [np.median, np.sum],
                              'Продажи, руб': np.sum,
                              'Повторение заказа': np.sum,
                              'Маржинальная прибыль': np.sum
                        
                               })
df_sales.head()

Маржинальная прибыль Повторение заказа Продажи, руб  \
                                sum               sum          sum   
Факты.Товар ID                                                       
PROD0000001               -28160.70                 6     631978.5   
PROD0000002               963721.00                32    5318764.0   
PROD0000003                32200.55                18      76755.0   
PROD0000004              -266380.65                17    3621384.0   
PROD0000005                31874.40                18     150984.0   

               Продажи, шт        
                    median   sum  
Факты.Товар ID                    
PROD0000001            2.5  12.0  
PROD0000002            3.5  51.0  
PROD0000003            2.0  23.0  
PROD0000004            4.0  29.0  
PROD0000005            5.0  24.0

In [47]:
newname=df_sales.columns.map('_'.join)
newname
df_sales.columns=newname
df_sales = df_sales.reset_index()
df_sales.head()

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum"
0,PROD0000001,-28160.70,6,631978.5,2.5,12.0
1,PROD0000002,963721.00,32,5318764.0,3.5,51.0
2,PROD0000003,32200.55,18,76755.0,2.0,23.0
3,PROD0000004,-266380.65,17,3621384.0,4.0,29.0
4,PROD0000005,31874.40,18,150984.0,5.0,24.0


In [48]:
df_sales['Продажи, шт_sum']
total_sale=df_sales['Продажи, руб_sum'].sum()
df_sales['Доля']=df_sales['Продажи, руб_sum']/total_sale * 100
df_sales['Доля'].sum()
df_sales = df_sales.sort_values(by=('Продажи, руб_sum'), ascending=False)
df_sales=df_sales.assign(sum_d=df_sales['Доля'].cumsum())

df_sales.loc[(df_sales['sum_d'] <= 80), 'ABC'] = 'A'
df_sales.loc[(df_sales['sum_d'] > 80) & (df_sales['sum_d'] <= 95), 'ABC'] = 'B'
df_sales.loc[(df_sales['sum_d'] > 95), 'ABC'] = 'C'

df_sales

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC
1388,PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845,A
450,PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071,A
1411,PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673,A
483,PROD0000488,1.746230e-10,15,10935288.0,5.0,39.0,0.952170,5.814843,A
1176,PROD0001187,1.116753e+06,33,9911739.5,3.0,37.0,0.863046,6.677889,A
...,...,...,...,...,...,...,...,...,...
1838,PROD0001871,1.578500e+03,1,3850.0,2.0,2.0,0.000335,99.999089,C
1854,PROD0001887,1.193400e+03,1,3536.0,2.0,2.0,0.000308,99.999396,C
1747,PROD0001779,1.555200e+03,3,3240.0,1.0,1.0,0.000282,99.999679,C
1675,PROD0001703,1.411200e+03,1,2880.0,2.0,2.0,0.000251,99.999929,C


In [49]:
df_sales['Стоимость, руб'] = df_sales['Продажи, руб_sum']/df_sales['Продажи, шт_sum']
df_sales['Продажи в следующем периоде']= (df_sales['Продажи, шт_sum'] + df_sales['Продажи, шт_median']) *df_sales['Стоимость, руб']
total_sale_next=df_sales['Продажи в следующем периоде'].sum()

total_sale_next

1372983059.734433

In [50]:
df_sales['Доля_будущая']=df_sales['Продажи в следующем периоде']/total_sale_next * 100
df_sales = df_sales.sort_values(by=('Продажи в следующем периоде'), ascending=False)
df_sales = df_sales.assign(sum_d_next=df_sales['Доля_будущая'].cumsum())
df_sales['Доля_будущая'].sum()

df_sales.loc[(df_sales['sum_d_next'] <= 80), 'ABC_next'] = 'A'
df_sales.loc[(df_sales['sum_d_next'] > 80) & (df_sales['sum_d_next'] <= 95), 'ABC_next'] = 'B'
df_sales.loc[(df_sales['sum_d_next'] > 95), 'ABC_next'] = 'C'

df_sales.loc[(df_sales['ABC'] != df_sales['ABC_next']), 'Изменение класса'] = 1
df_sales.loc[(df_sales['ABC'] == df_sales['ABC_next']), 'Изменение класса'] = 0

df_sales['Изменение класса'].sum()

df_sales.loc[(df_sales['sum_d'] <= 80), 'class'] = '0'
df_sales.loc[(df_sales['sum_d'] > 80) & (df_sales['sum_d'] <= 95), 'class'] = '1'
df_sales.loc[(df_sales['sum_d'] > 95), 'class'] = '2'

df_sales.loc[(df_sales['sum_d_next'] <= 80), 'class_next'] = '0'
df_sales.loc[(df_sales['sum_d_next'] > 80) & (df_sales['sum_d_next'] <= 95), 'class_next'] = '1'
df_sales.loc[(df_sales['sum_d_next'] > 95), 'class_next'] = '2'

df_sales

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC,"Стоимость, руб",Продажи в следующем периоде,Доля_будущая,sum_d_next,ABC_next,Изменение класса,class,class_next
1388,PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845,A,1.539996e+06,3.695989e+07,2.691941,2.691941,A,0.0,0,0
1411,PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673,A,1.886540e+06,2.263848e+07,1.648854,4.340795,A,0.0,0,0
450,PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071,A,4.427965e+05,1.505508e+07,1.096523,5.437318,A,0.0,0,0
483,PROD0000488,1.746230e-10,15,10935288.0,5.0,39.0,0.952170,5.814843,A,2.803920e+05,1.233725e+07,0.898572,6.335891,A,0.0,0,0
1629,PROD0001657,2.047488e+06,6,9187447.5,4.0,12.0,0.799980,9.126346,A,7.656206e+05,1.224993e+07,0.892213,7.228103,A,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,PROD0001257,1.050000e+03,12,4200.0,2.5,5.0,0.000366,99.997704,C,8.400000e+02,6.300000e+03,0.000459,99.998575,C,0.0,2,2
1108,PROD0001119,2.220000e+02,13,4884.0,1.5,6.0,0.000425,99.996103,C,8.140000e+02,6.105000e+03,0.000445,99.999020,C,0.0,2,2
801,PROD0000808,1.702000e+03,2,4048.0,2.5,5.0,0.000352,99.998056,C,8.096000e+02,6.072000e+03,0.000442,99.999462,C,0.0,2,2
1675,PROD0001703,1.411200e+03,1,2880.0,2.0,2.0,0.000251,99.999929,C,1.440000e+03,5.760000e+03,0.000420,99.999882,C,0.0,2,2


#### Далее предложены варианты проведения ABC-анализа с помощью методов машинного обучения:

Импорт требуемых модулей:

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, classification_report
from sklearn.metrics import confusion_matrix

Очистка данных от "шума" (лишних столбцов):

In [59]:
dataf = df_sales.copy()
dataf = dataf.drop(columns=['Факты.Товар ID', 'ABC', 'ABC_next', 'Продажи в следующем периоде', 'Доля_будущая','sum_d', 'sum_d_next', 'Изменение класса'])
dataf

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,"Стоимость, руб",class,class_next
1388,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,1.539996e+06,0,0
1411,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,1.886540e+06,0,0
450,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,4.427965e+05,0,0
483,1.746230e-10,15,10935288.0,5.0,39.0,0.952170,2.803920e+05,0,0
1629,2.047488e+06,6,9187447.5,4.0,12.0,0.799980,7.656206e+05,0,0
...,...,...,...,...,...,...,...,...,...
1245,1.050000e+03,12,4200.0,2.5,5.0,0.000366,8.400000e+02,2,2
1108,2.220000e+02,13,4884.0,1.5,6.0,0.000425,8.140000e+02,2,2
801,1.702000e+03,2,4048.0,2.5,5.0,0.000352,8.096000e+02,2,2
1675,1.411200e+03,1,2880.0,2.0,2.0,0.000251,1.440000e+03,2,2


Подготовка обучающей и тестовой выборок:

In [71]:
x = dataf.drop(columns = 'class_next')
y = dataf['class_next']
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, shuffle=True)

1. Случайный лес **RandomForestClassifier**:

In [73]:
sc = StandardScaler()
X_train = sc.fit_transform(x)
X_train_RF, X_test_RF, Y_train_RF, Y_test_RF = train_test_split(x, y, test_size=0.2, shuffle=True)

model_RandomForest = RandomForestClassifier()
model_RandomForest.fit(X_train_RF, Y_train_RF)
display(model_RandomForest.score(X_train_RF, Y_train_RF))

y_predict_RF = model_SVM.predict(X_test_RF)
print(confusion_matrix(Y_test_RF, y_predict_RF))
print(classification_report(Y_test_RF, y_predict_RF))

1.0

[[ 81   1   0]
 [  4 103   1]
 [  0   0 183]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97        82
           1       0.99      0.95      0.97       108
           2       0.99      1.00      1.00       183

    accuracy                           0.98       373
   macro avg       0.98      0.98      0.98       373
weighted avg       0.98      0.98      0.98       373



2. Метод опорных векторов **SVM**:

In [66]:
model_SVM = SVC(kernel='linear')
model_SVM.fit(X_train, Y_train)

y_predict_SVM = model_SVM.predict(X_test)
print(confusion_matrix(Y_test, y_predict_SVM))
print(classification_report(Y_test, y_predict_SVM))

[[ 92   0   0]
 [  3 100   0]
 [  0   3 175]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        92
           1       0.97      0.97      0.97       103
           2       1.00      0.98      0.99       178

    accuracy                           0.98       373
   macro avg       0.98      0.98      0.98       373
weighted avg       0.98      0.98      0.98       373



3. Дерево решений **DecisionTreeClassifier**:

In [68]:
model_DecisionTree = DecisionTreeClassifier()
model_DecisionTree.fit(X_train, Y_train)

y_predict_DT = model_DecisionTree.predict(X_test)
print(confusion_matrix(Y_test, y_predict_DT))
print(classification_report(Y_test, y_predict_DT))

[[ 90   2   0]
 [  2  95   6]
 [  0   4 174]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        92
           1       0.94      0.92      0.93       103
           2       0.97      0.98      0.97       178

    accuracy                           0.96       373
   macro avg       0.96      0.96      0.96       373
weighted avg       0.96      0.96      0.96       373



Модели были построены на основе разработанных в ходе выполнения алгоритма с лекции фичей.  
Наименьший показатель accuracy из трех моделей показал метод DecisionTreeClassifier, при этом показатель более 90, что уже является превосходным результатом.  
В то же время такое значение точности наталкивает на мысли о переобучении модели.